In [28]:
import tabula
import pandas as pd

In [39]:
# Replace 'bank_statement.pdf' with the actual path to your PDF file
pdf_file_path = 'path_to_mpesa_statement'

In [40]:
def data_cleaning(data_frame):
    # Dropping the unnecessary columns
    data_frame = data_frame.drop(data_frame.columns[7:], axis=1)
    data_frame = data_frame.drop(data_frame.columns[[3]], axis=1)
    #Dropping NA rows in the receipt no. column
    data_frame = data_frame.dropna(subset="Receipt No.")
    #converting completion time to date object
    data_frame["Completion Time"] = pd.to_datetime(data_frame["Completion Time"])
    #replacing \r in the details column with space
    data_frame["Details"] =  data_frame["Details"].str.replace(r"\r"," ",regex=True)
    #replacing the minus sign in the withdrawn column
    data_frame["Withdrawn"] = data_frame["Withdrawn"].str.replace("-","")
    #converting NAN to zero
    data_frame= data_frame.fillna(0)
    return data_frame

In [41]:
def extract_tables_from_pdf(pdf_file):
    # Read the tables from the PDF file
    tables = tabula.read_pdf(pdf_file, pages="all", multiple_tables=True)

    # Combine all tables except the first one into a single DataFrame
    # We start at table 2 to ignore the summary table
    combined_df = pd.concat(tables[2:], ignore_index=True)
    combined_df = data_cleaning(combined_df)
    return combined_df

In [42]:
combined_data_frame = extract_tables_from_pdf(pdf_file_path)
combined_data_frame

,Receipt No.,Completion Time,Details,Paid In,Withdrawn,Balance
0,NEE4WCCJUO,2019-05-14 12:18:33,Customer Transfer of Funds Charge,0,11.00,0.00
1,NEE4WCCJUO,2019-05-14 12:18:33,Customer Transfer Fuliza M-Pesa to 25479858860...,0,227.00,-227.00
2,NEE5WCB09V,2019-05-14 12:16:50,Receive International Zero Rated Transfer From...,914.00,0,914.00
3,NEE5WCAYST,2019-05-14 12:16:50,OD Loan Repayment to 232323 - M-PESA Overdraw,0,914.00,0.00
4,NED5W14995,2019-05-13 20:24:36,OverDraft of Credit Party,17.00,0,0.00
...,...,...,...,...,...,...
680,ND29X07UKJ,2019-04-02 10:34:23,Funds received from 254711643521 - SAOKE ALLAN...,50.00,0,50.00
681,NCM7O7CMDJ,2019-03-22 09:13:13,OD Loan Repayment to 232323 - M-PESA Overdraw,0,50.00,0.00
682,NCM5O7CNQN,2019-03-22 09:13:12,Funds received from 254711643521 - SAOKE ALLAN...,50.00,0,50.00
683,NC369EOZUY,2019-03-03 22:52:34,OD Loan Repayment to 232323 - M-PESA Overdraw,0,"1,117.00",0.00


In [43]:
combined_data_frame.dtypes

Receipt No.                object
Completion Time    datetime64[ns]
Details                    object
Paid In                    object
Withdrawn                  object
Balance                    object
dtype: object